# Blondy's Mountain Weather Forecast

In light of the discontinuation of the Avalanche Canada Mountain Weather Forecast (ACMWF), I took it upon myself to recreate the graphics with alternative means. For debugging, suggestions, etc, contact me at Andrew.Loeppky@Gmail.com

In [1]:
from herbie import Herbie
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd

from matplotlib.patches import Rectangle
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pytz
import os, shutil

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ WARNING: Unable to create config file               │
 │      /Users/andrew/.config/herbie/config.toml        │
 │ Herbie will use standard default settings.           │
 │ Consider setting env variable HERBIE_CONFIG_PATH.    │
 ╰──────────────────────────────────────────────────────╯



In [4]:
# get current model run (for GPDS, model runs are 0 and 12z)
# https://eccc-msc.github.io/open-data/msc-data/nwp_gdps/readme_gdps-datamart_en/#levels
tstep = range(3,243,3)

data_dir = "/Users/andrew/data/gdps/"
fig_dir = "/Users/andrew/Repos/BMWF/plots/"

#recent = pd.Timestamp.utcnow().floor("12h").tz_localize(None) - pd.Timedelta(hours=12)
recent = pd.Timestamp("now").floor("12h")
recent

Timestamp('2024-11-25 12:00:00')

In [ ]:
# loop through forecast times (0 to 240h, in 3h increments)
# clear the plot folder
for filename in os.listdir(fig_dir):
    file_path = os.path.join(fig_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

for fct in tstep:
    
    # clear the data folder
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
        

    # Get the necessary fields and parse into xarrays

    store = []
    # total cloud cover, precipitation rate
    for var, lev in zip(["TCDC", "PRATE"], ["SFC_0", "SFC_0"]):
        print(var, lev)
        _ds = Herbie(
            recent,
            model="gdps",
            fxx=fct,
            variable=var,
            level=lev,
        ).xarray()

        # if the variable name is unknown, assign it
        if "unknown" in _ds:
            _ds = _ds.rename({"unknown":var})

        store.append(_ds)

    # get 1000mb geopotential height and merge it
    hstore = []
    for var, lev in zip(["HGT", "HGT"], ["ISBL_500", "ISBL_1000"]):
        _ds = Herbie(
                recent,
                model="gdps",
                fxx=fct,
                variable=var,
                level=lev,
            ).xarray()

        # if the variable name is unknown, assign it
        if "unknown" in _ds:
            _ds = _ds.rename({"unknown":var})

        hstore.append(_ds)
    store.append(xr.concat(hstore, dim="isobaricInhPa"))
    ds = xr.merge(store)

    # slice out only the domain we care about
    ds = ds.sel(longitude=slice(-195, -97), latitude=slice(30, 73))
    
    # convert precip from kg/m3/s to mm/6h
    ds["prate"] *= (3600 * 6) 

    # calculate 1000 - 500mb thickness
    ds["thick"] = ds.gh.loc[{"isobaricInhPa":500}] - ds.gh.loc[{"isobaricInhPa":1000}] 
    
    # make a figure
    fig = plt.figure(figsize=(15,15))
    fig.tight_layout()
    ax = fig.add_subplot(1, 1, 1, 
                         position=(0,0,1,1), 
                         projection=ccrs.LambertConformal(central_longitude=-150, central_latitude=39.0,))
    ax.set_extent([-170, -117, 40, 63])

    # basemap
    ax.add_feature(feature.LAND)
    ax.add_feature(feature.STATES)
    ax.add_feature(feature.OCEAN)
    ax.add_feature(feature.COASTLINE)

    # plot height contours
    ht = ax.contour(
        ds.longitude,
        ds.latitude,
        ds.gh.loc[{"isobaricInhPa":500}],
        colors="k",
        transform=pc,
        levels=range(0, 6000, 50),
    )
    ax.clabel(ht, inline=True, fontsize=9)
    
    # plot thickness contours
    th_cmap = mpl.colors.ListedColormap(['pink',
                                         'blue',
                                         'orange',
                                        ])
    th_bounds = [4800, 5100, 5400, 5700]
    th_norm = mpl.colors.BoundaryNorm(th_bounds, th_cmap.N)
    ht = ax.contour(
        ds.longitude,
        ds.latitude,
        ds.thick,
        cmap=th_cmap, 
        norm=th_norm,
        linestyles="dashed",
        transform=pc,
        levels=range(4000, 6000, 60),
    )
    ax.clabel(ht, inline=True, fontsize=9)

    # plot cloud cover
    cc_cmap = mpl.colors.ListedColormap(['#DCDCDC',
                                         '#808080'])
    cc_bounds = [60, 80, 100]
    cc_norm = mpl.colors.BoundaryNorm(cc_bounds, cc_cmap.N)
    cc = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds.TCDC.where(ds.TCDC >= 60),
        cmap=cc_cmap,
        norm=cc_norm,
        antialiased=True,
        transform=pc,
    )

    # plot precip rate
    pr_cmap = mpl.colors.ListedColormap(['#90EE90',
                                         '#ADFF2F', 
                                         '#3CB371', 
                                         '#2E8B57', 
                                         '#FFFF00',
                                         '#FFA500', 
                                         '#B22222', 
                                         '#8B008B'])
    pr_bounds = [1, 2, 5, 10, 20, 30, 50, 75, 100]
    pr_norm = mpl.colors.BoundaryNorm(pr_bounds, pr_cmap.N)
    pr = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds.prate.where(ds.prate >= 1),
        cmap=pr_cmap,
        norm=pr_norm,
        antialiased=True,
        transform=pc,
    )

    # labels and such

    # configure text to match AC style
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["font.weight"] = "heavy"

    # shade out legend regions, AC style
    ax.add_patch(Rectangle((0, 0), 0.12, 0.85, transform=ax.transAxes, color="steelblue", alpha=0.8))
    ax.add_patch(Rectangle((0, 0.85), 1, 1, transform=ax.transAxes, color="steelblue", alpha=0.8))

    # precip colorbar
    fig.text(0.12, 0.65, "6hr Pcpn", color="white", size=10)
    fig.text(0.12, 0.64, "(mm)", color="white", size=10)
    pr_cbar = fig.colorbar(
                            pr,
                            cmap=pr_cmap, 
                            norm=pr_norm,
                            ax=ax,
                            spacing='uniform',
                            shrink=0.25,
                            location="left",
                            aspect=10,
                            anchor=(0.15,0.55),
                            pad=-0.15
                           )
    pr_cbar.ax.tick_params(labelsize=10, 
                           labelcolor="white", 
                           length=0, 
                           labelright=True,
                           labelleft=False,
                          ) 

    # cloud cover colorbar
    fig.text(0.12, 0.40, "Cloud Cover", color="white", size=10)
    fig.text(0.12, 0.39, "(%)", color="white", size=10)
    cc_cbar = fig.colorbar(
                            cc,
                            cmap=cc_cmap, 
                            norm=cc_norm,
                            ax=ax,
                            spacing='uniform',
                            shrink=0.125,
                            location="left",
                            aspect=5,
                            anchor=(0.15,0.25),
                            pad=-0.18
                           )
    cc_cbar.ax.tick_params(labelsize=10, 
                           labelcolor="white", 
                           length=0, 
                           labelright=True,
                           labelleft=False,
                          ) 

    # timestamps
    the_date = pd.Timestamp(ds.valid_time.values).tz_localize('UTC').tz_convert('America/Vancouver')
    fmt_date = the_date.strftime('%a. %m-%d-%y \n%H:%M %p')
    fig.text(0.5, 0.69, fmt_date, color="white", size=20)
    fig.text(0.12, 0.26, f"{ds.model.upper()} {pd.Timestamp(ds.time.values).strftime("%H")} Z", color="white", size=10)

    # title
    fig.text(0.35, 0.69, '500 mb\nHeight', color="white", size=20)

    # save and close the figure
    fig.savefig(f"{fig_dir}BMWF_{the_date.strftime('%m-%d-%y-%H-%M')}")
    fig.clf()
    

TCDC SFC_0
✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F03 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/gdps/20241125]
PRATE SFC_0
✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F03 ┊ GRIB2 @ msc ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F03 ┊ GRIB2 @ msc ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F03 ┊ GRIB2 @ msc ┊ IDX @ None


...

/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


PRATE SFC_0
✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F222 ┊ GRIB2 @ msc ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=gdps ┊ product=15km/grib2/lat_lon ┊ 2024-Nov-25 12:00 UTC F222 ┊ GRIB2 @ msc ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(
